In [1]:
from edgar import set_identity
import pandas as pd
import time

In [2]:
# must be set to comply with SEC regulations

set_identity("Neal Lockhart nhl3388@uncw.edu")

[13:51:15] INFO     Identity of the Edgar REST client set to [Neal Lockhart nhl3388@uncw.edu]           ]8;id=856508;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.12/site-packages/edgar/core.py\core.py]8;;\:]8;id=475829;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.12/site-packages/edgar/core.py#161\161]8;;\

## Filings: 10-k

### Markdown

In [32]:
from edgar import *

In [33]:
filings = Company("NOW").get_filings(form="10-K").latest(2)

In [34]:
md = filings[0].sections()

In [ ]:
def get_risk_sections(companies: list):

    data = {
        "ticker": [],
        "chunked_risk": []
    }
    
    for company in companies:
        time.sleep(.1)
        tenk = Company(company).get_filings(form="10-K").latest(1)
        raw_chunked_tenk = tenk.sections()
        index_start = None
        index_end = None
        
        for i in range(len(raw_chunked_tenk)):
            if re.search(r"item 1a.\s*risk factors", raw_chunked_tenk[i].lower()):
                index_start=i
            elif re.search(r"item 1b.\s*unresolved", raw_chunked_tenk[i].lower()):
                index_end=i
                break

        if index_start is None or index_end is None:
            print("Warning: Could not find risk section for", company)
            continue
        
        risks = raw_chunked_tenk[index_start:index_end]

        for i in range(len(risks)):
            risks[i] = risks[i].lower()
            risks[i] = risks[i].replace("\n"," ")
            risks[i] = risks[i].replace("•", " ")
            risks[i] = risks[i].replace("table of contents","")
            
            data['ticker'].append(company)
            data['chunked_risk'].append(risks[i])

    return pd.DataFrame(data)

In [44]:
companies = ['HUBS', 'SNOW', 'CRWD', 'ZS', 'NET', 'AMD', 'NVDA', 'BASE', 'NOW', 'S']

df = get_risk_sections(companies)

In [45]:
df.head()

,ticker,chunked_risk
0,HUBS,item 1a. risk factors
1,HUBS,an investment in our common stock involves a h...
2,HUBS,risks related to our business and strategy we...
3,HUBS,"we derive, and expect to continue to derive, a..."
4,HUBS,risks related to employee matters if we canno...


In [58]:
df.to_csv("../data/risk_sections.csv", index=False)

In [53]:
lens = df['chunked_risk'].apply(lambda x: len(x.split()))
max_len = lens.idxmax()

In [56]:
max(lens)

13845

In [57]:
df.iloc[max_len]['ticker']

'S'